<a href="https://colab.research.google.com/github/nicoschmidt/DigitalCentones/blob/master/SmartTexts2019/Practicum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Texts 2019 Practicum May 27th:
# Christus Patiens and the opportunities for Digital Reading

Welcome to the Smart Texts practicum! Today we will investigate the possibilities of computer-aided intertextual reading.
In this session you will get to know some techniques to handle a digital version of the Greek cento Christus Patiens from late antiquity by Gregorius Nazianzenus. You can have a look at a digital version, an edition by Johann Georg Brambs from 1885 in the Scaife Viewer: https://scaife.perseus.org/reader/urn:cts:greekLit:tlg2022.tlg003.opp-grc1:1.1-1.30/

It is also available as TEI formatted xml file on GitHub (along with many other texts!) in the repository of the ***OpenGreekAndLatin*** project: https://github.com/OpenGreekAndLatin/First1KGreek/blob/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml

We will make use of this formatted text for analyzing it with our software. As you have learned in the seminar today, a Cento is a patchwork poem made of many parts, lines or single words from other (source) texts. In the case of Christus Patiens the source texts to a large part are chosen from tragedies by Euripides, such as Medea, Bacchae and others.

We will use the alignment algorithm discussed during today's seminar - Smith-Waterman - to find for each line of the Christus Patiens the corresponding (matching) text in the source tragedies. For time reasons and because of limited availability of the source text as formatted xml files, we will restrict our analysis to the first 100 lines of the Christus Patiens and match them with the first 48 lines of Euripides' *Medea*.

As with most algorithms, the Smith-Waterman algorithm comes with some parameters which needs to be tuned in order to produce meaningful results. In the coming exercises you will play around with those parameters to tweak the algorithm. You will compare your results to the matches found by an expert researcher in this field (Tuillier 1969).

If you look for a more advanced programming exercise, you can even work on extensions of the alignment program - during this practicum or continue your work afterwards in a student project.


# 1) Get to know the data



## Prepare code
Steps:
- get code and data files from GitHub
- import code and other libs

In [2]:
!([ -d DigitalCentones ]) && rm -r DigitalCentones
!git clone https://github.com/nicoschmidt/DigitalCentones

from DigitalCentones.smith_waterman import SmithWaterman
from DigitalCentones.alignment import *
from IPython.display import HTML
import numpy as np

Cloning into 'DigitalCentones'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 22), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (53/53), done.


## Load Christus Patiens and Medea text files
Steps:
- copy xml file of Christus Patiens from GitHub repository ***First1KGreek*** by the ***OpenGreekAndLatin*** project
- parse the xml file
- remove preface lines
- load first 48 lines from Medea as plain text (no xml available yet :( )
---


### Exercise 1.1: Get familiar with the text
Have a look at the [xml file](https://github.com/OpenGreekAndLatin/First1KGreek/blob/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml).
It contains all lines of the poem with lots of meta information, in the header as well as along each page:
- editorial information
- information about online edition
- line numbers
- footnotes

We parse the lines like this:
```
<l n="1">Εἴθ᾿ ὤφελ’ ἐν λειμῶνι μηδ’ ἕρπειν ὄφις,</l>

```
Have a look at the function [load_xml(fname) on line 1667 of alignment.py](https://github.com/nicoschmidt/DigitalCentones/blob/3c1df2ce56f5d4776be395dd62773e70b715cd88/alignment.py#L167).

---


### Exercise 1.2: Get familiar with the data structures
The lines are loaded in a list-like structure.

While parsing, some pre-processing is done that will help with finding alignments:
- punctuation removed
- lower case
- diacritics removed


In [3]:
# get Christus Patiens xml file (if not present)
!([ ! -f tlg2022.tlg003.opp-grc1.xml ]) && wget https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml
lines_chr_pat = load_xml('tlg2022.tlg003.opp-grc1.xml')
for i in range(30, len(lines_chr_pat)):
    lines_chr_pat[i].no = -29+i # hack to re-number lines
lines_chr_pat = lines_chr_pat[30:100]
print('Christus Patiens:', lines_chr_pat[:3],'\n\n')

lines_medea = load_plain_text('DigitalCentones/data/Medea-48.txt', 'Medea', 'Medea', 'Euripides')
print('Medea:', lines_medea[:3], '\n\n')

--2019-05-22 19:14:18--  https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403903 (394K) [text/plain]
Saving to: ‘tlg2022.tlg003.opp-grc1.xml’

tlg2022.tlg003.opp- 100%[===================>] 394.44K  --.-KB/s    in 0.09s   

2019-05-22 19:14:18 (4.41 MB/s) - ‘tlg2022.tlg003.opp-grc1.xml’ saved [403903/403903]

Christus Patiens: [work_id:            urn:cts:greekLit:tlg2022.tlg003.opp-grc1
idx:                30
no:                 1
speaker:            ΘΕΟΤΟΚΟΣ.
text_raw:           Εἴθ᾿ ὤφελ’ ἐν λειμῶνι μηδ’ ἕρπειν ὄφις,
text:               εἴθ ὤφελ ἐν λειμῶνι μηδ ἕρπειν ὄφις
text_no_diacritics: ειθ ωφελ εν λειμωνι μηδ ερπειν οφις
text_lemma:     




---

**Print all characters with diacritics**



In [4]:
# print all characters in the corpus:
alphabet = np.unique([c for lines in [lines_chr_pat, lines_medea] for line in lines for c in line.text])
print('ID, Character, Name')
for a in alphabet:
    print('{} "{}" {}'.format(hex(ord(a)), a, unicodedata.name(a, 'not defined')))

ID, Character, Name
0x20 " " SPACE
0x3ac "ά" GREEK SMALL LETTER ALPHA WITH TONOS
0x3ad "έ" GREEK SMALL LETTER EPSILON WITH TONOS
0x3ae "ή" GREEK SMALL LETTER ETA WITH TONOS
0x3af "ί" GREEK SMALL LETTER IOTA WITH TONOS
0x3b1 "α" GREEK SMALL LETTER ALPHA
0x3b2 "β" GREEK SMALL LETTER BETA
0x3b3 "γ" GREEK SMALL LETTER GAMMA
0x3b4 "δ" GREEK SMALL LETTER DELTA
0x3b5 "ε" GREEK SMALL LETTER EPSILON
0x3b6 "ζ" GREEK SMALL LETTER ZETA
0x3b7 "η" GREEK SMALL LETTER ETA
0x3b8 "θ" GREEK SMALL LETTER THETA
0x3b9 "ι" GREEK SMALL LETTER IOTA
0x3ba "κ" GREEK SMALL LETTER KAPPA
0x3bb "λ" GREEK SMALL LETTER LAMDA
0x3bc "μ" GREEK SMALL LETTER MU
0x3bd "ν" GREEK SMALL LETTER NU
0x3be "ξ" GREEK SMALL LETTER XI
0x3bf "ο" GREEK SMALL LETTER OMICRON
0x3c0 "π" GREEK SMALL LETTER PI
0x3c1 "ρ" GREEK SMALL LETTER RHO
0x3c2 "ς" GREEK SMALL LETTER FINAL SIGMA
0x3c3 "σ" GREEK SMALL LETTER SIGMA
0x3c4 "τ" GREEK SMALL LETTER TAU
0x3c5 "υ" GREEK SMALL LETTER UPSILON
0x3c6 "φ" GREEK SMALL LETTER PHI
0x3c7 "χ" GREEK SMALL L

**Print all characters without diacritics**

In [5]:
alphabet_no_diacritics = np.unique([c for lines in [lines_chr_pat, lines_medea] for line in lines for c in line.text_no_diacritics])
print('ID, Character, Name')
for a in alphabet_no_diacritics:
    print('{} "{}" {}'.format(hex(ord(a)), a, unicodedata.name(a, 'not defined')))


ID, Character, Name
0x20 " " SPACE
0x3b1 "α" GREEK SMALL LETTER ALPHA
0x3b2 "β" GREEK SMALL LETTER BETA
0x3b3 "γ" GREEK SMALL LETTER GAMMA
0x3b4 "δ" GREEK SMALL LETTER DELTA
0x3b5 "ε" GREEK SMALL LETTER EPSILON
0x3b6 "ζ" GREEK SMALL LETTER ZETA
0x3b7 "η" GREEK SMALL LETTER ETA
0x3b8 "θ" GREEK SMALL LETTER THETA
0x3b9 "ι" GREEK SMALL LETTER IOTA
0x3ba "κ" GREEK SMALL LETTER KAPPA
0x3bb "λ" GREEK SMALL LETTER LAMDA
0x3bc "μ" GREEK SMALL LETTER MU
0x3bd "ν" GREEK SMALL LETTER NU
0x3be "ξ" GREEK SMALL LETTER XI
0x3bf "ο" GREEK SMALL LETTER OMICRON
0x3c0 "π" GREEK SMALL LETTER PI
0x3c1 "ρ" GREEK SMALL LETTER RHO
0x3c2 "ς" GREEK SMALL LETTER FINAL SIGMA
0x3c3 "σ" GREEK SMALL LETTER SIGMA
0x3c4 "τ" GREEK SMALL LETTER TAU
0x3c5 "υ" GREEK SMALL LETTER UPSILON
0x3c6 "φ" GREEK SMALL LETTER PHI
0x3c7 "χ" GREEK SMALL LETTER CHI
0x3c8 "ψ" GREEK SMALL LETTER PSI
0x3c9 "ω" GREEK SMALL LETTER OMEGA


## Load all Matches found by Tuillier 1969
As a baseline we compare our alignment results later on to the matches found by an expert researcher in the field, Andre Tuillier, who, in his edition of the Christus Patiens from 1969, listed down all the corresponding lines he found relevant. Thanks to Lena, who analized these correspondences during her PhD thesis, we have these correspondences as a [machine-readable text file](https://github.com/nicoschmidt/DigitalCentones/blob/master/data/ChristusPatiens_Tulliers_matches.csv) and can load them here.

In [10]:
# Input file is a text file with one header row and remaining rows structured as
# '<target_line_number>,<source_work_number>,<source_work_line_number>...\n'
line_to_line_map = np.loadtxt('DigitalCentones/data/ChristusPatiens_Tulliers_matches.csv', int, delimiter=',', skiprows=1, usecols=[0,1,2])

# These are all matched works Toullier found
source_work_names = ['Agamemnon',
                     'Prometheus',
                     'Alkestis',
                     'Andromache',
                     'Bakchen',
                     'Hekabe',
                     'Helena',
                     'Hippolytos',
                     'Iphigenie in Aulis',
                     'Iphigenie in Tauris',
                     'Medea',
                     'Orestes',
                     'Ph\"onikerinnen',
                     'Rhesos',
                     'Troerinnen',
                     'Ilias',
                     'Alexandra']

# store map as a dictionary with target_line_number as keys and tuples (<source_work_id>, <source_work_number>) as values 
line_to_line_map = dict([(target_line_no,(source_work_names[source_work_id-1], source_line_id)) for target_line_no, source_work_id, source_line_id in line_to_line_map])
for k in list(line_to_line_map)[:50]:
    print('{}: {}'.format(k, line_to_line_map[k]))

1: ('Medea', 1)
2: ('Medea', 3)
3: ('Medea', 6)
4: ('Medea', 20)
5: ('Hekabe', 1123)
6: ('Medea', 8)
8: ('Medea', 9)
14: ('Medea', 10)
15: ('Medea', 11)
17: ('Prometheus', 1027)
25: ('Alexandra', 151)
32: ('Medea', 14)
33: ('Medea', 15)
34: ('Medea', 13)
37: ('Medea', 16)
38: ('Medea', 17)
39: ('Agamemnon', 764)
40: ('Troerinnen', 605)
41: ('Troerinnen', 620)
42: ('Troerinnen', 621)
43: ('Medea', 20)
45: ('Prometheus', 1027)
46: ('Medea', 25)
47: ('Medea', 26)
50: ('Hippolytos', 450)
51: ('Medea', 21)
52: ('Medea', 22)
53: ('Medea', 34)
54: ('Medea', 35)
55: ('Medea', 36)
56: ('Medea', 56)
57: ('Medea', 57)
58: ('Medea', 58)
59: ('Medea', 59)
61: ('Hekabe', 736)
65: ('Agamemnon', 611)
66: ('Agamemnon', 612)
71: ('Agamemnon', 587)
72: ('Agamemnon', 588)
73: ('Agamemnon', 589)
75: ('Agamemnon', 593)
76: ('Agamemnon', 591)
77: ('Troerinnen', 747)
78: ('Troerinnen', 748)
79: ('Agamemnon', 594)
80: ('Agamemnon', 595)
81: ('Agamemnon', 596)
82: ('Agamemnon', 597)
88: ('Rhesos', 63)
90: ('Rhe

# 2) Compare alignments

## The expert's results
First we go through the lines listed by Tuillier to find exact line-to-line alignments and visualize them.

With our automatic alignment search later on, we seek to get as close as possible to these matches **without knowing anything about ancient Greek tragedies!** 

In [0]:
# find matches by searching in given line-to-line map
smith_waterman = SmithWaterman(match_score=2,
                               mismatch_score=-1,
                               gap_score=-1,
                               n_max_alignments=1,
                               min_score_treshold=0)
alignments_outfile = 'alignments_word-based_Toullier'
target_lines = lines_chr_pat
source_lines_dict = {lines_medea[0].work_id:lines_medea}
alignments = find_line_to_line_alignments(target_lines, source_lines_dict, smith_waterman.align, 3, lambda line:line.text_no_diacritics.split(' '), line_to_line_map)
save_alignments_html(alignments, target_lines, source_lines_dict, alignments_outfile, lambda line:line.text_raw.split(' '), 4)
HTML(filename='./'+ alignments_outfile + '.html')

searching all verses for alignments...
0/70 (  0%)
10/70 ( 14%)
20/70 ( 29%)
30/70 ( 43%)
40/70 ( 57%)
50/70 ( 71%)
60/70 ( 86%)


Target Line ID,Target Line No.,Target Line Text,Source Line Text,Source Line No.,Source Text ID,Alignment Score
30,1,"Εἴθ᾿ ὤφελ’ ἐν λειμῶνι μηδ’ ἕρπειν ὄφις,",Εἴθ’ ὤφελ’ Ἀργοῦς μὴ διαπτάσθαι σκάφος,1,Medea,4
31,2,μηδ’ ἐν νάπαισι τοῦδ’ ὑφεδρεύειν δράκων,μηδ’ ἐν νάπαισι Πηλίου πεσεῖν ποτε,3,Medea,6
32,3,"ἀγκυλομήτης· οὐ γὰρ ἂν πλευρᾶς φῦμα,",Πελίαι μετῆλθον. οὐ γὰρ ἂν δέσποιν’ ἐμὴ,6,Medea,6
33,4,"μήτηρ γένους δύστηνος ἠπατημένη,",,,,
34,5,"τόλμημα τολμᾶν παντάτολμον ἀνέτλη,",,,,
35,6,"ἔρνους ἔρωτι θυμὸν ἐκπεπληγμένη,",ἔρωτι θυμὸν ἐκπλαγεῖσ’ Ἰάσονος·,8,Medea,4
36,7,"θεώσεως πεισθεῖσα τυχεῖν αὐτόθεν,",,,,
37,8,οὐδ’ ἂν φαγεῖν πείσασα καρποῦ τὸν πόσιν,οὐδ’ ἂν κτανεῖν πείσασα Πελιάδας κόρας,9,Medea,5
38,9,τοῦ μηδὲ συμφέροντος αὐτίκα σφίσι,,,,
39,10,"λειμῶνος ἐξῴκιστο τοῦ πανολβίου,",,,,


## Find character-based line-to-line alignments with diacritics


In [0]:
smith_waterman = SmithWaterman(match_score=2,
                               mismatch_score=-1,
                               gap_score=-1,
                               n_max_alignments=1,
                               min_score_treshold=0)
line_form_func = lambda line:line.text # we use the lower-case text with diacritics
alignments_outfile = 'alignments_character-based_with_diacritics'
target_lines = lines_chr_pat
source_lines_dict = {lines_medea[0].work_id:lines_medea}
alignments = find_line_to_line_alignments(target_lines, source_lines_dict, smith_waterman.align, 3, line_form_func)
save_alignments_html(alignments, target_lines, source_lines_dict, alignments_outfile, line_form_func)
HTML(filename='./'+ alignments_outfile + '.html')

searching all verses for alignments...
0/70 (  0%)
10/70 ( 14%)
20/70 ( 29%)
30/70 ( 43%)
40/70 ( 57%)
50/70 ( 71%)
60/70 ( 86%)


Target Line ID,Target Line No.,Target Line Text,Source Line Text,Source Line No.,Source Text ID,Alignment Score
30,1,εἴθ ὤφελ ἐν λειμῶνι μηδ ἕρπειν ὄφις,εἴθ ὤφελ ἀργοῦς μὴ διαπτάσθαι σκάφος,1,Medea,18
31,2,μηδ ἐν νάπαισι τοῦδ ὑφεδρεύειν δράκων,μηδ ἐν νάπαισι πηλίου πεσεῖν ποτε,3,Medea,27
32,3,ἀγκυλομήτης οὐ γὰρ ἂν πλευρᾶς φῦμα,πελίαι μετῆλθον οὐ γὰρ ἂν δέσποιν ἐμὴ,6,Medea,22
33,4,μήτηρ γένους δύστηνος ἠπατημένη,μήδεια δ ἡ δύστηνος ἠτιμασμένη,20,Medea,21
34,5,τόλμημα τολμᾶν παντάτολμον ἀνέτλη,οἷον πατρώιας μὴ ἀπολείπεσθαι χθονός,35,Medea,9
35,6,ἔρνους ἔρωτι θυμὸν ἐκπεπληγμένη,ἔρωτι θυμὸν ἐκπλαγεῖσ ἰάσονος,8,Medea,31
36,7,θεώσεως πεισθεῖσα τυχεῖν αὐτόθεν,τμηθεῖσα πεύκη μηδ ἐρετμῶσαι χέρας,4,Medea,12
37,8,οὐδ ἂν φαγεῖν πείσασα καρποῦ τὸν πόσιν,οὐδ ἂν κτανεῖν πείσασα πελιάδας κόρας,9,Medea,34
38,9,τοῦ μηδὲ συμφέροντος αὐτίκα σφίσι,κακῶν νέα γὰρ φροντὶς οὐκ ἀλγεῖν φιλεῖ,48,Medea,14
39,10,λειμῶνος ἐξῴκιστο τοῦ πανολβίου,γήμας κρέοντος παῖδ ὃς αἰσυμνᾶι χθονός,19,Medea,10


## Find character-based line-to-line alignments without diacritics

In [0]:
smith_waterman = SmithWaterman(match_score=2,
                               mismatch_score=-1,
                               gap_score=-1,
                               n_max_alignments=1,
                               min_score_treshold=0)
line_form_func = lambda line:line.text_no_diacritics
alignments_outfile = 'alignments_character-based_without_diacritics'
target_lines = lines_chr_pat
source_lines_dict = {lines_medea[0].work_id:lines_medea}
alignments = find_line_to_line_alignments(target_lines, source_lines_dict, smith_waterman.align, 3, line_form_func)
save_alignments_html(alignments, target_lines, source_lines_dict, alignments_outfile, line_form_func, 4)
HTML(filename='./'+ alignments_outfile + '.html')

searching all verses for alignments...
0/70 (  0%)
10/70 ( 14%)
20/70 ( 29%)
30/70 ( 43%)
40/70 ( 57%)
50/70 ( 71%)
60/70 ( 86%)


Target Line ID,Target Line No.,Target Line Text,Source Line Text,Source Line No.,Source Text ID,Alignment Score
30,1,ειθ ωφελ εν λειμωνι μηδ ερπειν οφις,ειθ ωφελ αργους μη διαπτασθαι σκαφος,1,Medea,18
31,2,μηδ εν ναπαισι τουδ υφεδρευειν δρακων,μηδ εν ναπαισι πηλιου πεσειν ποτε,3,Medea,35
32,3,αγκυλομητης ου γαρ αν πλευρας φυμα,πελιαι μετηλθον ου γαρ αν δεσποιν εμη,6,Medea,23
33,4,μητηρ γενους δυστηνος ηπατημενη,μηδεια δ η δυστηνος ητιμασμενη,20,Medea,27
34,5,τολμημα τολμαν παντατολμον ανετλη,τον παντα συντηκουσα δακρυοις χρονον,25,Medea,15
35,6,ερνους ερωτι θυμον εκπεπληγμενη,ερωτι θυμον εκπλαγεισ ιασονος,8,Medea,32
36,7,θεωσεως πεισθεισα τυχειν αυτοθεν,ουδ αν κτανειν πεισασα πελιαδας κορας,9,Medea,13
37,8,ουδ αν φαγειν πεισασα καρπου τον ποσιν,ουδ αν κτανειν πεισασα πελιαδας κορας,9,Medea,37
38,9,του μηδε συμφεροντος αυτικα σφισι,εχθραν τις αυτη καλλινικον αισεται,45,Medea,15
39,10,λειμωνος εξωκιστο του πανολβιου,ανδρων αριστεων οι το παγχρυσον δερος,5,Medea,14


## Find word-based line-to-line alignments without diacritics

In [0]:
smith_waterman = SmithWaterman(match_score=2,
                               mismatch_score=-1,
                               gap_score=-1,
                               n_max_alignments=1,
                               min_score_treshold=0)
line_form_func = lambda line:line.text_no_diacritics.split(' ')
alignments_outfile = 'alignments_word-based_without_diacritics'
target_lines = lines_chr_pat
source_lines_dict = {lines_medea[0].work_id:lines_medea}
alignments = find_line_to_line_alignments(target_lines, source_lines_dict, smith_waterman.align, 3, line_form_func)
save_alignments_html(alignments, target_lines, source_lines_dict, alignments_outfile, lambda line:line.text_raw.split(' '), 4)
HTML(filename='./'+ alignments_outfile + '.html')

searching all verses for alignments...
0/70 (  0%)
10/70 ( 14%)
20/70 ( 29%)
30/70 ( 43%)
40/70 ( 57%)
50/70 ( 71%)
60/70 ( 86%)


Target Line ID,Target Line No.,Target Line Text,Source Line Text,Source Line No.,Source Text ID,Alignment Score
30,1,"Εἴθ᾿ ὤφελ’ ἐν λειμῶνι μηδ’ ἕρπειν ὄφις,",Εἴθ’ ὤφελ’ Ἀργοῦς μὴ διαπτάσθαι σκάφος,1,Medea,4
31,2,μηδ’ ἐν νάπαισι τοῦδ’ ὑφεδρεύειν δράκων,μηδ’ ἐν νάπαισι Πηλίου πεσεῖν ποτε,3,Medea,6
32,3,"ἀγκυλομήτης· οὐ γὰρ ἂν πλευρᾶς φῦμα,",Πελίαι μετῆλθον. οὐ γὰρ ἂν δέσποιν’ ἐμὴ,6,Medea,6
33,4,"μήτηρ γένους δύστηνος ἠπατημένη,",,,,
34,5,"τόλμημα τολμᾶν παντάτολμον ἀνέτλη,",,,,
35,6,"ἔρνους ἔρωτι θυμὸν ἐκπεπληγμένη,",ἔρωτι θυμὸν ἐκπλαγεῖσ’ Ἰάσονος·,8,Medea,4
36,7,"θεώσεως πεισθεῖσα τυχεῖν αὐτόθεν,",,,,
37,8,οὐδ’ ἂν φαγεῖν πείσασα καρποῦ τὸν πόσιν,οὐδ’ ἂν κτανεῖν πείσασα Πελιάδας κόρας,9,Medea,5
38,9,τοῦ μηδὲ συμφέροντος αὐτίκα σφίσι,,,,
39,10,"λειμῶνος ἐξῴκιστο τοῦ πανολβίου,",,,,


# Incorporating morphological information
- load morphological data base from GitHub
- Map all words to their lemmatized form
- Find alignments using the lemmas

In [0]:
# download files if not present
!([ ! -f MorpheusGreek1-319492.xml ]) && wget https://raw.githubusercontent.com/gcelano/MorpheusGreekUnicode/master/MorpheusGreek1-319492.xml
!([ ! -f MorpheusGreek319493-638984.xml ]) && wget https://raw.githubusercontent.com/gcelano/MorpheusGreekUnicode/master/MorpheusGreek319493-638984.xml
!([ ! -f MorpheusGreek638985-958476.xml ]) && wget https://raw.githubusercontent.com/gcelano/MorpheusGreekUnicode/master/MorpheusGreek638985-958476.xml
morph_xml_fnames = ['MorpheusGreek1-319492.xml',
                    'MorpheusGreek319493-638984.xml',
                    'MorpheusGreek638985-958476.xml']

# load data base
morph_dict, max_id = load_morphology(morph_xml_fnames, False)
print('{} lemmas loaded'.format(max_id))
m = max_id
max_id = add_lemma_ids(lines_chr_pat, morph_dict, max_id, False)
print('{} lemmas added by Christus Patiens'.format(max_id - m))
m = max_id
max_id = add_lemma_ids(lines_medea, morph_dict, max_id, False)
print('{} lemmas added by Medea'.format(max_id - m))
print('{} total lemmas'.format(max_id))



36488 lemmas loaded
υφεδρευειν
παντατολμον
ερνους
θεωσεως
εξωκιστο
πανολβιου
κατακριθεισα
δυσκοιτιας
μογοστοκουσ
εμπαρειμενη
ιδρωτ
στονοις
εξοχους
αλθανειν
αγαθοτητι
βροτωθηναι
τλησαι
δαλων
μαιμασσει
νητρεκως
συμεων
τηλεσκοποις
προβλεπων
μεγεστη
παρφασιν
συμφρονουσα
ητιμωμενη
κλαουσα
αφερτατων
πρωτοπημονος
υποκλιναντος
ορωσ
φευγουσ
κορειης
34 lemmas added by Christus Patiens
πελιαι
μηδεια
επλευσ
ιωλκιας
εκπλαγεισ
ιασονος
πελιαδας
ξυμφερουσ
ιασονι
αισυμναι
υφεισ
επαιρουσ
προδουσ
πασχουσ
εσβασ
εστρωται
αισεται
17 lemmas added by Medea
36539 total lemmas


# 3) Advanced exercises and extensions

Now that you got the basic ideas and a feeling for how the algorithm behaves and how parameters affect the outputs, here are some suggestions how to further improve the software.
## Exercise 3.1: Word count statistics
Modify the alignment scoring by introducing a weighted matching score scheme, where frequent (but potentially uninteresting) words get lower match scores and rare, (but potentially interesting words) get higher match scores.

## Exercise 3.2: 

